**Prediksi Quantity Menggunakan 7 Model Regresi**

In [ ]:
#PACKAGES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

# DATA
file_path = 'data.xlsx' # <- PILIH DATA DISINI
data = pd.read_excel(file_path)

# // Buat DF dengan period D-X terbalik
def process_pn(data, pn_to_filter):
    filtered_data = data[data['P/N'] == pn_to_filter]
    formatted_data = []
    for index, row in filtered_data.iterrows():
        reversed_quantities = [row[f'D-{i}'] for i in range(1, 13)][::-1]
        for i, quantity in enumerate(reversed_quantities, start=1):
            formatted_data.append({
                'P/N': row['P/N'],
                'Period': i,
                'Quantity': quantity
            })

    formatted_df = pd.DataFrame(formatted_data)

    return formatted_df


# PILIH PN DISINI // Hanya pilih 1 PN untuk diprediksi
pn_to_filter = 205401  # <--- nih yang ini (APABILA MEMILIKI HURUF PAKAI '...')
formatted_df = process_pn(data, pn_to_filter)
display(formatted_df)


In [ ]:

# MOVING AVERAGE

# Calculate the sum of the first 12 periods
last_12_sum = formatted_df['Quantity'].iloc[:12].sum() 

# Add a new column for the moving average and set all values to NaN
formatted_df['MA_Prediction'] = np.nan

# Calculate the moving average for the first 12 periods
for i in range(1, 13):
    formatted_df.loc[formatted_df['Period'] == i, 'MA_Prediction'] = formatted_df['Quantity'].iloc[:i].sum() / i

# Predict the moving average for the 13th period
ma_13th_period = last_12_sum / 13

# Add the 13th period to the DataFrame
formatted_df = pd.concat([
    formatted_df,
    pd.DataFrame({'Period': [13], 'MA_Prediction': [ma_13th_period], 'Quantity': [np.nan]})
], ignore_index=True)

# Display the updated DataFrame
display(formatted_df)




In [ ]:

# WEIGHTED MOVING AVERAGE
# Define a function to calculate the exponential weighted moving average
def calculate_exponential_weighted_moving_average(values, alpha=0.2):
    return values.ewm(alpha=alpha, adjust=False).mean()

# Set the alpha (smoothing factor) for the weighted moving average
alpha = 0.3

# Calculate the weighted moving average and store it in a new column
formatted_df['Smoothed_Weighted_Prediction'] = calculate_exponential_weighted_moving_average(
    formatted_df['Quantity'], alpha
)
display(formatted_df)

In [ ]:


# REGRESSION LINEAR
# Remove rows with missing values in the 'Quantity' column
valid_data = formatted_df.dropna(subset=['Quantity'])

# Prepare the input (X) and output (y) data for linear regression
X = valid_data[['Period']]
y = valid_data['Quantity']

# Create and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Use the model to predict values for all periods and store them in a new column
formatted_df['Linear_Regression_Prediction'] = model.predict(formatted_df[['Period']])

# Display the linear regression predictions
display(formatted_df[['Period','Linear_Regression_Prediction']])


In [ ]:

# POLINOM DERAJAT 2
degree = 2
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)
poly_model = LinearRegression()
poly_model.fit(X_poly, y)
X_all_poly = poly.transform(formatted_df[['Period']])
formatted_df['Polynomial_Regression_Prediction_Degree_2'] = poly_model.predict(X_all_poly)
display(display(formatted_df[['Period','Polynomial_Regression_Prediction_Degree_2']]))


In [ ]:

# POLINOM DERAJAT 3
degree = 3
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)
poly_model = LinearRegression()
poly_model.fit(X_poly, y)
X_all_poly = poly.transform(formatted_df[['Period']])
formatted_df['Polynomial_Regression_Prediction_Degree_3'] = poly_model.predict(X_all_poly)
display(formatted_df[['Period','Polynomial_Regression_Prediction_Degree_3']])



In [ ]:

# SINGLE EXPONENTIAL SMOOTHING
alpha_ses = 0.8  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.8   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)
def apply_ses(values, alpha=alpha_ses):
    ses_model = SimpleExpSmoothing(values).fit(smoothing_level=alpha, optimized=False)
    return ses_model.fittedvalues


# DOUBLE EXPONENTIAL SMOOTHING
def apply_des(values, alpha=alpha_ses, beta=beta_des):
    des_model = ExponentialSmoothing(values, trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
    return des_model.fittedvalues

formatted_df['SES_Prediction'] = apply_ses(formatted_df['Quantity'], alpha=alpha_ses)
formatted_df['DES_Prediction'] = apply_des(formatted_df['Quantity'], alpha=alpha_ses, beta=beta_des)

display(formatted_df[['Period','SES_Prediction','DES_Prediction']])


In [ ]:

# Calculate RMSE (Root Mean Square Error) and R2 (R-squared) metrics for various prediction models
results = []  # Initialize an empty list to store the evaluation results

# Iterate over the list of prediction columns in the DataFrame
for column in ['MA_Prediction', 'Smoothed_Weighted_Prediction', 'Linear_Regression_Prediction', 
               'Polynomial_Regression_Prediction_Degree_2', 'Polynomial_Regression_Prediction_Degree_3', 
               'SES_Prediction', 'DES_Prediction']:
    # Drop rows with NaN values in 'Quantity' or the current prediction column
    valid_data = formatted_df.dropna(subset=['Quantity', column])
    
    # Skip evaluation if no valid data is available for the current prediction column
    if valid_data.empty:
        print(f"Skipping {column} due to lack of valid data.")
        continue
    
    try:
        # Calculate RMSE between the actual 'Quantity' and predicted values in the column
        rmse = np.sqrt(mean_squared_error(valid_data['Quantity'], valid_data[column]))
        # Calculate R2 score to measure how well the predictions fit the actual data
        r2 = r2_score(valid_data['Quantity'], valid_data[column])
        # Append the results (model name, RMSE, R2) to the results list
        results.append({'Model': column, 'RMSE': rmse, 'R2': r2})
    except Exception as e:
        # Print an error message if an exception occurs during evaluation
        print(f"Error evaluating {column}: {e}")

# Convert the results list into a DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Sort the DataFrame by R2 scores in descending order and display the results
print(results_df.sort_values(by='R2', ascending=False).to_string(index=False))


In [ ]:

# Identify the best model based on the highest R2 score
best_model_row = results_df.loc[results_df['R2'].idxmax()]  # Find the row with the maximum R2 value
best_model_name = best_model_row['Model']  # Extract the name of the best model
best_model_r2 = best_model_row['R2']  # Extract the highest R2 value

# Retrieve the prediction for the 13th period using the best model
next_period_prediction = formatted_df.loc[formatted_df['Period'] == 13, best_model_name].values[0]

# Print the results: best model name, its R2 score, and the prediction for the 13th period
print(f"Best Model: {best_model_name}")
print(f"Best R-squared: {best_model_r2}")
print(f"Prediction for 13th Period: {next_period_prediction}")


In [ ]:

#PLOT SEMUA GRAFIK
plt.figure(figsize=(12, 8))
plt.plot(formatted_df['Period'], formatted_df['Quantity'], label='Data Historis', marker='o', color='black')
plt.plot(formatted_df['Period'], formatted_df['MA_Prediction'], label='Prediksi MA', linestyle='--', color='orange')
plt.plot(formatted_df['Period'], formatted_df['Smoothed_Weighted_Prediction'], label='Prediksi WMA', linestyle='--', color='blue')
plt.plot(formatted_df['Period'], formatted_df['Linear_Regression_Prediction'], label='Prediksi Regresi Linear', linestyle='--', color='green')
plt.plot(formatted_df['Period'], formatted_df['Polynomial_Regression_Prediction_Degree_2'], label='Prediksi Regresi Polinomial Derajat 2', linestyle='--', color='red')
plt.plot(formatted_df['Period'], formatted_df['Polynomial_Regression_Prediction_Degree_3'], label='Prediksi Regresi Polinomial Derajat 3', linestyle='--', color='purple' )
plt.plot(formatted_df['Period'], formatted_df['SES_Prediction'], label='Prediksi SES', linestyle='--', color='magenta' )
plt.plot(formatted_df['Period'], formatted_df['DES_Prediction'], label='Prediksi DES', linestyle='--', color='yellow' )
plt.xlabel('Period')
plt.ylabel('Quantity')
plt.title(f'Prediksi Quantity untuk P/N {pn_to_filter}')
plt.legend()
plt.grid(False)
plt.show()

# PLOT GRAFIK PALING BAGUS
plt.figure(figsize=(10, 6))
plt.plot(formatted_df['Period'], formatted_df['Quantity'], label='Actual Quantity', marker='o', color='black')
plt.plot(formatted_df['Period'], formatted_df[best_model_name], label=f'Best Model: {best_model_name}', linestyle='-', color='blue')
plt.xlabel('Period')
plt.ylabel('Quantity')
plt.title(f'Prediksi menggunakan {best_model_name} untuk 1 bulan kedepan')
plt.legend()
plt.grid(False)
plt.show()
